In [6]:
import numpy as np
import scipy.stats as stats


In [15]:
# data col's: Time(s) Height(mm) Diameter(mm) Lenght(mm) Observer(0|1)
D = np.loadtxt('../data/measurements.txt')


# prepare data for analysis
v = 2000 # 20 ml target for volume 
g = 9820 # acceleration due to gravity in mm/s
nu = 1.53 # kinematic viscosity, mu/rho, mm^2/s
M = []
for [t, h, d, l] in D[:, 0:4]:
    M.append([(v/t)/((l**2)*((l*g)**0.5)), (h/l), (d/l), (nu/(l*((l*g)**0.5)))])
M = np.array(M)

# Model specification matrix, replace col 1 with ones, take natural log of values 
X = np.log(M[:, 0:4])
X[:, 0] = 1
X = np.c_[X, D[:, 4]] # add observer indicator variable 

# Realization of response vector 
y = np.log(M[:, 0])

for i, yi in enumerate(y):
    print(f'{yi} = {X[i]}')

-19.350271769594475 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-19.34576439920819 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-19.34576439920819 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-13.48477830320871 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-13.48701508317948 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-13.503633333783831 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-13.431395073455182 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-13.44696414997021 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-13.458032240161268 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-17.82583838981178 = [  1.          -0.58434432  -6.45368247 -14.32356028   0.        ]
-17.856834850183112 = [  1.          -0.58434432  -6.45368247 -14.32356028   0.        ]
-17.719296328421095 = [  1.

### Fysisk modell

$\frac{\langle Q\rangle}{l^2\sqrt{lg}}=\Phi(\frac{h}{l}, \frac{d}{l}, \frac{\mu}{\rho l\sqrt{lg}})$

där $l$ är ett rörs längd, $d$ dess diameter, $h$ är vattenreservoarens höjd över röret, $g$ är accelerationen pga gravitation, $\mu$ är vätskans viskositet och $\rho$ dess densitet. $Q$ är den uppmätta flödeshastigheten. Detta har räknats fram med Buckinghams Teorem, en dimensionsanalysteknik från fysiken.

Genom att ansätta nya variabler istället för kvoterna och anta en potensform kan uttrycket skrivas om på formen

\begin{equation}
q = \alpha k^ar^bm^c.
\end{equation}

Linearisering genom att ta $\ln$ i båda led:

\begin{equation*}
    \ln q = \ln \alpha + a \ln k + b \ln r + c \ln m.
\end{equation*}
Vi kan se detta som formen:
\begin{equation}
    \mu_{Y|x_1,x_2,x_3} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3
\end{equation}
alltså
\begin{equation*}
    \mathbf{Y} = X\boldsymbol\beta + \mathbf{E},
\end{equation*}
där $X$ är modellspecifikations matrisen. $\mathbf{E}$ är vektorn med stokastiska fel med avseende på väntevärdet och $\mathbf{Y}$ är en respons vektor för $Y$ över hela stickprovet. 

Normalform: 
\begin{equation*}
    (X'X)\mathbf{b} = X'\mathbf{y},
\end{equation*}
där $\mathbf{y}$ är vektorn av observeraded värden i respons vektorn $\mathbf{Y}$ och $\mathbf{b}$ är den sökta vektorn med de uppskattade modell parametrarna. OLS för $\boldsymbol\beta$ blir,
\begin{equation}
    \boldsymbol{\hat{\beta}} = \mathbf{b} = (X'X)^{-1}X'\mathbf{y}.
\end{equation}

### Regression och statistik 

In [12]:
# Multiple linear regression, least square estimates for B^
b = np.linalg.inv(X.T @ X) @ X.T @ y # (X'X)^-1X'y this is the normal equation for least squares
b

array([-4.88635418,  0.87000771,  3.60315069, -0.75188958,  0.0168805 ])

Vi räknar ut ett antal statistiska värden:

$v$ antalet parametrar i modellen (number of features)

$n$ stickprovets storlek (number of samples, sample size)

SSE Sum of Squared Errors aka Residual Sum of Squares

\begin{equation}
\sum_{i=1}^n[y_i - (b_0 + b_1 x_{1i} + ... + b_k x_{ki})]²
\end{equation}

Vilket alltså betyder att vi drar ifrån högerledet från vänsterledet på varje rad, kvadrerar och sedan lägger ihop alla resultaten.

Variansen

$\sigma^2 = \frac{\mathrm{SSE}}{n-v-1}$ 

Standarddeviation

$S = \sqrt{\sigma^2} = \sigma$

SSR Regression Sum of Squares
\begin{equation}
b_0 \sum_{i=1}^n{y_i} + b_1 \sum_{i=1}^n{x_{1i}y_i} + ... + b_v \sum_{i=1}^n{x_{vi} y_i} - \frac{(\sum_{i=1}^n{y_i})²}{n}
\end{equation}

$S_{yy}$ total varians (i responsvektorn Y)

\begin{equation}
\sum_{i=1}^n{y_i²}-\frac{(\sum_{i=1}^n{y_i})²}{n}
\end{equation}



In [16]:
# Sum of Squared Errors, deviation and variance 
v = len(b)-1 # degrees of freedom
n = len(M) # number of observations from data M matrix

# Sum of Squared Errors
SSE = np.sum(np.square(y - (X @ b)))

# Variance of the residuals
var  = SSE/(n-v-1)

# Standard deviation of the residuals
std = np.sqrt(var)

# Regression sum of squares, total variance 
SSR = np.sum(b*(X.T @ y)) - (np.square(np.sum(y)/n)) # regression sum of squares
Syy = np.sum(np.square(y)) - (np.square(np.sum(y)/n)) # total variance

## Signifikanstest
Är regressionen signifikant? Eller är alla $\beta_i$ = 0?

Ett sorts rimlighetstest -- om vi inte klarar detta test är det ingen idé att försöka utröna något ur resultaten.

Vår hypotes $H_0$ är alltså att alla $\beta_i$ är noll.

Om detta är sant följer nedanstående uttryck en F-distribution med $k$ och $n-k-1$ frihetsgrader. Detta värde läses av i en tabell.

\begin{equation}
\frac{\mathrm{SSR}/k}{S²}
\end{equation}
där $S²$ är en uppskattning av $\sigma²$.

Om vi får ett väldigt litet värde är alltså hypotesen falsk, och minst en parameter förklarar någon del av datan.

In [18]:
# Significance regression testm, test rejects for large values
sig_stat = (SSR/v)/var

# P-value for H0 in sig_stat is very small, reject H0 that no variables are significant
p_sig = stats.f.sf(sig_stat, v, n-v-1)

p_sig

0.0

## Förklaringsgrad( coefficient of multiple determination)

När vi vet att reg är signifikant är nästa fråga, **hur sig**? Hur mycket av datan **täcks av modellen**, dvs **till vilken grad förklarar modellen datan**?

\begin{equation*}
R^2 = \frac{\mathrm{SSR}}{S_{yy}}
\end{equation*}

Detta är ett värde mellan 0 och 1.

In [20]:
RSQ = SSR/Syy # R^2 value, proportion of variance explained by the model
RSQ

Rsq = 1 - (SSE/Syy) # R^2 value, proportion of variance explained by the model
Rsq

0.9999707077549623

## Modellraffinering 

Men vilka parametrar är relevanta? Och vi har ju en varibel som bara indikerar vem som gjort observationen. Vad gör vi med den?

- Vi kan testa variableran en och en!
Test statistika: 

Test statistika:
\begin{equation}
\frac{\hat{\beta_i}}{S\sqrt{c_{ii}}}
\end{equation}
där $\mathbb{c}_{ii}$ är elementet på rad och kolumn $(i, i)$ en särskild matris, kallad varians-kovarians matrisen.

Om hypotesen att variabeln är 0 är sann följer t-stat en $T_{n-k-1}$ distribution.
Om värdet är väldigt stort eller väldigt lite avfärdar vi hypotesen och variabeln __är__ relevant.

- D V S, om __observatörs-indikator__ är relevant så har vi ett observatörs-bias in data!



### Varians/Kovarians matris:

Vi härleder inte uttrycket, men på matrisform är det

\begin{equation}
\mathbb{c} = (X^TX)^{-1}\sigma²
\end{equation}

Variansen $\sigma²$ förekommer på diagonalen i denna matris, och parvis relationer med övriga parametrar (kovarians) fyller ut resten av matrisen. 

In [25]:
# Variance-covariance matrix
cov = np.linalg.inv(X.T @ X)*var

# is there no observer bias?
b4_stat = b[4]/ (std*np.sqrt(cov[4, 4])) # test statistic for observer bias

# Two sided test, reject H0 for small or large values
p_b4 = 2 * min(stats.t.cdf(b4_stat, n-v-1), stats.t.sf(b4_stat, n-v-1))
print(p_b4)

2.3422411110827324e-44


Se där! Det finns ett observatörsbias i datan. 

Vad gör vi då?

I själva verket har vi nu två modeller. Dessa modeller har samma lutning, men olika skärningspunkter!

För observatör A är modellen:
\begin{equation}
\mu_A = (\beta_0 + \beta_4) + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 
\end{equation}
och för B:
\begin{equation}
\mu_B = \beta_0  + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 
\end{equation}

Notera att regressionen har hittat ett värde för $\beta_4$:

In [27]:
print(b[4])

0.016880504116747522


In [28]:
# Split models according to observer 
modA = np.append(b[0] + b[4], b[1:4]) # model for observer A
modB = b[0:4] # model for observer B
Xr = X[:, 0:4] # model matrix for both observers


Så vilken model är bäst? 

In [38]:
# Normalize error around reg line
norm_a = (y - (Xr @ modA))/var # normalized error for observer A
norm_b = (y - (Xr @ modB))/var # normalized error for observer B

# Binding for chi^2 test, avoindin empty intervals an many low numbers 
# interarively found through trial and error
binsA, rangesA = np.histogram(norm_a, 48, (-20, 21))
p_A, _ = np.histogram(norm_a, 48, (-20, 21), density=True)

# chi^2 goodness of fit test
# -- rejects H0 for large values
chi2_statA = 0
for i in range(0, 48):
    chi2_statA += (np.square(binsA[i] - n*p_A[i]))/(n*p_A[i])

# p-value very large, do not goodness of fit
p_chi2A = stats.chi2.sf(chi2_statA, 48-1)

binsB, rangesB = np.histogram(norm_b, 48, (-20, 21))
p_B, _ = np.histogram(norm_b, 48, (-20, 21), density=True)

# chi^2 goodness of fit test
chi_statB = 0 # set to zero
for i in range(0, 48):
    chi_statB += np.divide(np.square(binsB[i] - n*p_B[i]), n*p_B[i]) # chi^2 test statistic

# p-value very large, do not reject goodness of fit
p_chi2B = stats.chi2.sf(chi_statB, 48-1)
print(p_chi2A, p_chi2B)


0.9999999986322857 0.9999999937061224


Modellerna är alltså likvärdiga bortom $3\sigma$ så skillnaden är i praktiken långt bortom de 99.7% av variansen som regressionen förklarar. Men tekniskt sett är modell A något bättre.

Om vi återgår till orginalformen och räknar ut det modellerade flödet för varje mätpunkt och jämför med det uppmätta flödet tillsammans med vår regressionslinje får vi följande graf:

![image](../assets/regression.png)